### Import libraries

In [1]:
import os # Operating system operations
import shutil as st # Copy operations
import numpy as np # Array and vector-matrix operations
import random # Shuffle datas
import pickle # Save and load history files
import cv2 # Resize images
import matplotlib.image as mping # Read RGB images
import seaborn # Confusion matrix display
import pandas as pd # Data reading
import matplotlib.pyplot as plt # Visualition
import skimage # Loading images
import keras_tuner as kt # Hyperparameter optimization


# Required packages for our ConvNet
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50, ResNet152,  DenseNet201, MobileNetV2, EfficientNetB0, EfficientNetB7
from tensorflow.keras.utils import Sequence
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from imblearn.over_sampling import RandomOverSampler
from imblearn.tensorflow import balanced_batch_generator
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, balanced_accuracy_score,  precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.utils import class_weight
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve,auc

# Required packages for extracting files, progression bar etc.
from tqdm.auto import tqdm
from io import BytesIO
from zipfile import ZipFile
from pathlib import Path
from shutil import copyfileobj
from tqdm.utils import CallbackIOWrapper
from os import fspath

### Downloading and extracting the dataset

The following section will download the necessarry dataset and the models. The 'Datas' folder contains:
 - Training set: Images for the training process (8512)
 - Validation set: Images for validation (751)
 - Test set: Images for testing (752)
 - Gaussian test set: Test set images with Gaussian noise
 - Poisson test set: Test set images with Poisson noise
 - test: Test images for check the CNN confidence

The models folder contains:
 - default models: The default models. They have been trained with transfer learning (all layers frozen, feature extraction)
 - optimized models: The optimized models. They have been trained with transfer learning (some layers were newly trained and optimized)
 - ensemble models: The final model with ensemble technique.

Helper function for creating directories, extracting files with progression bar.

In [2]:
def create_folders():
    
    models_dir                = 'new_models'
    metrics_dir               = 'metrics'
    metrics_visualization_dir = 'metrics_visualization'
    plots_dir                 = 'plots'
    heatmaps_dir              = 'heatmaps'
    roc_auc_dir               = 'ROC-AUC'
    hyperparameters_dir       = 'hyperparameters'
    
    try:
        os.mkdir(models_dir)
        os.mkdir(os.path.join(models_dir, 'default'))
        os.mkdir(os.path.join(models_dir, 'optimized'))
        
        os.mkdir(metrics_dir)
        os.mkdir(os.path.join(metrics_dir, 'default'))
        os.mkdir(os.path.join(metrics_dir, 'optimized'))
        os.mkdir(os.path.join(metrics_dir, 'ensemble'))
        os.mkdir(os.path.join(metrics_dir, 'poisson'))
        os.mkdir(os.path.join(metrics_dir, 'gauss'))
        os.mkdir(os.path.join(metrics_dir, 'FGSM'))

        os.mkdir(metrics_visualization_dir)
        os.mkdir(os.path.join(metrics_visualization_dir, 'default'))
        os.mkdir(os.path.join(metrics_visualization_dir, 'optimized'))
        os.mkdir(os.path.join(metrics_visualization_dir, 'ensemble'))

        os.mkdir(plots_dir)
        os.mkdir(os.path.join(plots_dir, 'default'))
        os.mkdir(os.path.join(plots_dir, 'optimized'))
        
        os.mkdir(heatmaps_dir)
        os.mkdir(os.path.join(heatmaps_dir, 'default'))
        os.mkdir(os.path.join(heatmaps_dir, 'optimized'))
        os.mkdir(os.path.join(heatmaps_dir, 'ensemble'))
        os.mkdir(os.path.join(heatmaps_dir, 'perturbation'))

        os.mkdir(roc_auc_dir)
        os.mkdir(os.path.join(roc_auc_dir, 'default'))
        os.mkdir(os.path.join(roc_auc_dir, 'optimized'))
        os.mkdir(os.path.join(roc_auc_dir, 'ensemble'))
        
        os.mkdir(hyperparameters_dir)
        
        print('Directories were successfully created!')
    except OSError as error:
        print(error)    

def extractall(fzip, dest, desc="Extracting"):
    
    """zipfile.Zipfile(fzip).extractall(dest) with progress"""
    dest = Path(dest).expanduser()
    with ZipFile(fzip) as zipf, tqdm(
        desc=desc, unit="B", unit_scale=True, unit_divisor=1024,
        total=sum(getattr(i, "file_size", 0) for i in zipf.infolist()),
    ) as pbar:
        for i in zipf.infolist():
            if not getattr(i, "file_size", 0):  # directory
                zipf.extract(i, fspath(dest))
            else:
                with zipf.open(i) as fi, open(fspath(dest / i.filename), "wb") as fo:
                    copyfileobj(CallbackIOWrapper(pbar.update, fi), fo)                   

Downloading the datasets. If it does not work, please download the datasets directly from the following links and copy into the code folder:
 - https://drive.google.com/file/d/1gCMmE9Wjm7YXkwRNW5wS85tim4z_m-br/view?usp=sharing
 - https://drive.google.com/file/d/1_V3xN3MoqjLSdbjFDalFRhQm5R29nAG7/view?usp=sharing

In [3]:
#https://drive.google.com/file/d/1gCMmE9Wjm7YXkwRNW5wS85tim4z_m-br/view?usp=sharing
#https://drive.google.com/file/d/1_V3xN3MoqjLSdbjFDalFRhQm5R29nAG7/view?usp=sharing

!gdown --id 1gCMmE9Wjm7YXkwRNW5wS85tim4z_m-br
!gdown --id 1_V3xN3MoqjLSdbjFDalFRhQm5R29nAG7

Access denied with the following error:


c:\users\habon\appdata\local\programs\python\python39\lib\site-packages\gdown\cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1gCMmE9Wjm7YXkwRNW5wS85tim4z_m-br 



Access denied with the following error:


c:\users\habon\appdata\local\programs\python\python39\lib\site-packages\gdown\cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1_V3xN3MoqjLSdbjFDalFRhQm5R29nAG7 



Extracting the downloaded datasets

In [ ]:
extractall('Datas.zip', '.')
extractall('models.zip', '.')

Creating directories for the models, metrics, heatmaps, plots etc.

In [ ]:
create_folders()

### Global variables

In [ ]:
# Categories, Path and sizes

multi_categories = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'] # The multiclass images categories

multi_train = './Datas/Multiclass/Training set'
multi_val = './Datas/Multiclass/Validation set'
multi_test = './Datas/Multiclass/Test set'

multiclass_directories = [multi_train, multi_val, multi_test]

multi_train_size = 8512
multi_val_size   = 751
multi_test_size  = 752

multi_sizes = [multi_train_size, multi_val_size, multi_test_size]

### Class for balancing the dataset and for the CNNs

DataBalancerGenerator class:
 - Creating a new, balanced dataset with data augmentation technique

SkinCancerCnn class:

 - The class for the CNNs. This class has all the necessary functions for preprocessing, training, testing, metrics, and visualization.

In [ ]:
#========================== Create a DataBalancerGenerator class, which help us to balancing the imbalanced dataset =========================#

class DataBalancerGenerator(Sequence): # inherited from Sequence

    # initialize the class members, and applying Data augmentation and Random oversampling
    def __init__(self, x, y, datagen, batch_size = 32):
    
        self.datagen    = datagen # ImageDataGenerator, which we use to Data augmentation
        self.batch_size = batch_size
        self.shape      = x.shape        
        
        datagen.fit(x)# compute quantities required for featurewise normalization  (std, mean, and principal components if ZCA whitening is applied)
        self.balanced_gen, self.steps_per_epoch = balanced_batch_generator(x.reshape(x.shape[0], -1),
                                                                           y           = to_categorical(y, num_classes = 7),
                                                                           sampler     = RandomOverSampler(),
                                                                           batch_size  = self.batch_size,
                                                                           keep_sparse = True) # balacing the dataset
    
    # determine the number of batches in the Sequence.
    def __len__(self):
        return self.steps_per_epoch
 

    # create batches and use Data augmentation
    def __getitem__(self, idx): # idx here is a mandatory parameter, but we don't use this.
    
        x_batch, y_batch = self.balanced_gen.__next__()
        x_batch = x_batch.reshape(-1, *self.shape[1:])
        
        return self.datagen.flow(x_batch, y_batch, batch_size = self.batch_size).next() # .flow returns an iterator, so we have to use next() method to get the batches with tuple type.

In [ ]:
class SkinCancerCnn:

    # initialize the class members
    def __init__(self, image_size,multi_learning_rate, multi_batch_size, multi_decrease_lr, multi_early_stop, multi_epochs_num):
    
        self.image_size           = image_size           # the given image size (the size that we want to use  to train a model in our ConvNet)
        self.multi_learning_rate  = multi_learning_rate  # the multiclass classification model learning rate
        self.multi_batch_size     = multi_batch_size     # the multiclass classification model batch size (defines the number of samples that will be propagated through the network.)
        self.multi_decrease_lr    = multi_decrease_lr    # the multiclass classification model decrease learning rate (when this is true, the learning rate will be decreasing)
        self.multi_early_stop     = multi_early_stop     # the multiclass classification model early stopping (when this is true, the training will stop when the validation loss won't decrease)
        self.multi_epochs_num     = multi_epochs_num     # the multiclass classification models epochs number (the number of complete passes through the training dataset)
        
    
    # load the images and create a numpy image array, which contains all the images that we need
    @staticmethod
    def create_image_arr(img_size, path, datasets, preprocesser, mode):
    
        images = []

        for dataset in datasets:
            images_list = os.listdir(path + '/' +  dataset)
            img_label   = datasets.index(dataset) # we need to map the labels to numbers(indexes 0-6)

            for img in images_list:
                my_image = mping.imread(path + '/' + dataset + '/' + img) # read the images
                my_image = cv2.resize(my_image, (img_size, img_size),) # resize the images
                images.append([my_image, img_label]) 

        if mode == 'train':
            random.shuffle(images) # we have to shuffle the images, because if we don't, then the model start to memorize them

        X = []
        y = []

        for matrix, label in images:
            X.append(matrix)
            y.append(label)

        X = np.array(X) # we need to map the list into a numpy array
        y = np.array(y) # the corresponding labels

        if mode == 'test':
            X = preprocesser(X) # Normalize data 

        print('Image array done with shape: ' + str(X.shape)) # the numpy array (matrix) shape

        return X,y # return with the image array and the corresponding labels

    
    # Load the images with ImageDataGenerator which allow us to preprocess them easily
    def create_multiclass_generators(self, X, y, train_path, val_path, multi_nums_arr, preprocesser):

        # we use this ImageDataGenerator for Data augmentation (because the dataset is pretty imbalanced and we have to correct this)
        datagen = ImageDataGenerator(preprocessing_function = preprocesser,
                                     rotation_range         = 180,
                                     width_shift_range      = 0.08,
                                     height_shift_range     = 0.08,
                                     zoom_range             = 0.065,
                                     horizontal_flip        = True,
                                     vertical_flip          = True,
                                     fill_mode              = 'nearest') 
        
        batch_size = self.multi_batch_size

        balanced_gen = DataBalancerGenerator(X, y, datagen, batch_size = self.multi_batch_size) # Oversampling and Data augmentation to get more samples 
        
        val_steps = multi_nums_arr[1] // batch_size

        # we will use this generator under the training session in order to evaluate the model
        val_datagen = ImageDataGenerator(preprocessing_function = preprocesser)

        val_gen = val_datagen.flow_from_directory(val_path,
                                                  target_size = (self.image_size,self.image_size),
                                                  batch_size  = batch_size,
                                                  class_mode  = "categorical") 

        return balanced_gen, val_gen, val_steps # return with the 2 generator (which basically an iterator type) and the corresponding steps (we will use that later in the model)
    
    
    # Create multiclass classification model with transfer learning
    def multi_class_model(self, train_gen, val_gen, val_steps, current_model, units, num_layers, dropout, globlayer, freeze_layers_num):
             
        # The pre-trained CNN
        conv_base = current_model(weights     = 'imagenet', # initialize the weights
                                  include_top = False, # we don't want to use the top layers ( we want to use our densely connected classifier)
                                  input_shape = (self.image_size, self.image_size, 3)) # the input shape( usually it's (224,224,3))
      
        # Our dataset is different to any subset of the imagenet dataset, therefore freezing will mean a decrease in accuracy
        # unfreezing some layers will allow us to optimize in the whole feature space, allowing to find better optima    

        if freeze_layers_num == 0:
            conv_base.trainable = False
        else:
            for layer in conv_base.layers[:freeze_layers_num]:
                layer.trainable = False 
        
        model = models.Sequential()
        model.add(conv_base)
        
        if globlayer:
            model.add(GlobalAveragePooling2D())

        model.add(layers.Flatten()) # flattens the 3D tensor of embeddings into a 2D tensor of shape 
        
        for i in range(num_layers):
            
            model.add(layers.Dense(units, activation = 'relu', kernel_regularizer = regularizers.l2(0.01))) # use Regularization and Dropout in order to reduce overfitting
            
            if dropout: 
                model.add(layers.Dropout(0.35))
                
        model.add(layers.Dense(7, activation = 'softmax')) # final layer with 7 output        
        
        # compile and run the model
        model.compile(loss      = 'categorical_crossentropy', # Multiclass, single-label classification
                      optimizer = Adam(learning_rate = self.multi_learning_rate), # use RMSprop optimizer with the given learning rate
                      metrics   = ["accuracy",
                                   tf.keras.metrics.Precision(name = 'precision'),
                                   tf.keras.metrics.Recall(name = 'recall')]) # we only care about accuracy 

        return model # return the model and the corresponding history (we will use these things later)

    
    
    # Create multiclass classification training process
    def multi_class_model_train(self, train_gen, val_gen, val_steps, current_model, units, num_layers, dropout, globlayer, freeze_layers_num, model_name):
    
        model = self.multi_class_model(train_gen,
                                       val_gen,
                                       val_steps,
                                       current_model,
                                       units,
                                       num_layers,
                                       dropout,
                                       globlayer,
                                       freeze_layers_num)
        
        callbacks = []
        
        # For TensorBoard visualisation
        #tensorboard = TensorBoard(log_dir = 'logs/{}'.format(model_name), histogram_freq = 1)
        #callbacks.append(tensorboard)
            
        # If the user want to decrease the learning rate under the training session
        if self.multi_decrease_lr:
            decrease_lr = ReduceLROnPlateau(monitor   = 'val_recall',
                                            factor    = 0.2,
                                            min_lr    = 0.0000001,
                                            patience  = 2,
                                            verbose   = 1,
                                            min_delta = 1e-6,
                                            mode      = 'max')
                                            
            callbacks.append(decrease_lr)
            
        # If the user want to use early stopping under the training session  
        if self.multi_early_stop:
            early_stop = EarlyStopping(monitor              = 'val_recall',
                                       min_delta            = 0,
                                       patience             = 3,
                                       verbose              = 0,
                                       mode                 = 'max',
                                       baseline             = None,
                                       restore_best_weights = True)
                                       
            callbacks.append(early_stop)

        history = model.fit(
                    train_gen,
                    callbacks        = callbacks,
                    epochs           = self.multi_epochs_num,
                    validation_data  = val_gen,
                    validation_steps = val_steps)

        return model, history # return the model and the corresponding history
    
    
    # Calculate and save the metrics
    @staticmethod
    def metrics_scores(y_true, y_pred, filename):
        # precision - The predictions are the baseline
        # recall - The grand truth labels are the baseline
        # f1-score - The harmonic mean of percision and recall (overall performance of the model)

        accuracy          = accuracy_score(y_true, y_pred) # How many we got right
        recall            = recall_score(y_true, y_pred, average = 'macro') # Truth labels are the baseline, All class[i] truth how many we got right
        precision         = precision_score(y_true, y_pred, average = 'macro') # All class[i] predictions how many we got right
        f1_score          = 2 * (precision * recall) / (precision + recall)
        balanced_accuracy = balanced_accuracy_score(y_true, y_pred)
        mcc               = matthews_corrcoef(y_true, y_pred)
        avg               = (balanced_accuracy + f1_score + mcc) / 3.0
        
        format_accuracy          = "{:.4f}".format(accuracy)
        format_precision         = "{:.4f}".format(precision)
        format_balanced_accuracy = "{:.4f}".format(balanced_accuracy)
        format_f1_score          = "{:.4f}".format(f1_score)
        format_mcc               = "{:.4f}".format(mcc)
        format_avg               = "{:.4f}".format(avg)
        
        acc_format = "{:.2f}".format(accuracy * 100)
        avg_format = "{:.2f}".format(avg * 100)
        
        # Save the metrics
        with open(filename, 'w') as file:

            file.write("Accuracy of the total model is {:.4f}".format(accuracy) + '\n')
            file.write("Precision of the total model is {:.4f}".format(precision) + '\n')
            file.write("Recall of the total model is {:.4f}".format(recall) + '\n')
            file.write("Balanced accuracy of the total model is {:.4f}".format(balanced_accuracy) + '\n')
            file.write("F1 score of the total model is {:.4f}".format(f1_score) + '\n')
            file.write("MCC of the total model is {:.4f}".format(mcc) + '\n')  
            file.write("AVG of the total model is {:.4f}".format(avg) + '\n')  
            file.write('\n\n\n')
            file.write(format_accuracy + '\t' + format_precision + '\t' + format_balanced_accuracy + '\t' + format_f1_score + '\t' + format_mcc + '\t' + format_avg)
            file.write('\n\n')
            file.write(acc_format)
            file.write('\n')
            file.write(avg_format)
        
        
        print(classification_report(y_true, y_pred))
        
    
    # Plot the confusion matrix   
    @staticmethod
    def confusion_matrix(y_true, y_pred, norm, filename):
        
        plt.rcParams["figure.figsize"] = (6,4)

        cm      = confusion_matrix(y_true = y_true, y_pred = y_pred, normalize = norm)
        ax      = plt.subplot()
        
        if norm == None:
            heatmap = seaborn.heatmap(cm, annot = True, cmap = "Blues", annot_kws = {"size" : 12}, fmt = 'd') 
        else:
            heatmap = seaborn.heatmap(cm, annot = True, cmap = "Blues", annot_kws = {"size" : 12}) 
        
        ax.set_xlabel('Predicted labels')
        ax.set_ylabel('True labels')
        ax.set_title('Confusion Matrix')
        ax.xaxis.set_ticklabels(['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'])
        ax.yaxis.set_ticklabels(['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'])
        ax.figure.savefig(filename, dpi = 400)        
        
    
    # Plot the training process 
    @staticmethod    
    def plot_model(history, filename):
        
        accuracy      = history.history['accuracy']
        val_accuracy  = history.history['val_accuracy']
        
        recall     = history.history['recall']
        val_recall = history.history['val_recall']

        loss     = history.history['loss']
        val_loss = history.history['val_loss']

        fig, ax    = plt.subplots(1, 3, figsize=(20, 8))
        ax         = ax.ravel()
        num_epochs = len(history.history['loss'])

        for i, met in enumerate(['accuracy', 'recall', 'loss']):
            ax[i].plot(history.history[met])
            ax[i].plot(history.history['val_' + met])
            ax[i].set_title('Model {}'.format(met))
            ax[i].set_xlabel('epochs')
            ax[i].set_xticks(range(1,num_epochs))
            ax[i].set_ylabel(met)
            ax[i].legend(['Training ' + met, 'Validation ' + met])

        plt.savefig(filename)  
        
    
    # Plot ROC-AUC curve
    @staticmethod        
    def plot_ROC_AUC_curve(y_true, y_pred, filename):

        # roc curve for classes
        fpr = {}
        tpr = {}
        thresh ={}
        roc_auc = dict()

        n_class = 7
        plt.rcParams["figure.figsize"] = (12,8)

        y_t = label_binarize(y_true, classes = np.unique(y_true))

        for i in range(n_class):    
            fpr[i], tpr[i], thresh[i] = roc_curve(y_t[:,i], y_pred[:,i])
            roc_auc[i] = auc(fpr[i], tpr[i])

            # plotting    
            plt.plot(fpr[i], tpr[i], linestyle='--', 
                     label='%s vs Rest (AUC=%0.2f)'%(SkinCancerCnn.map_labels(i),roc_auc[i]))

        plt.plot([0,1],[0,1],'b--')
        plt.xlim([0,1])
        plt.ylim([0,1.05])
        plt.title('Multiclass ROC curve')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive rate')
        plt.legend(loc='lower right')
        plt.savefig(filename)
        plt.show()
        
        
    # Map the indexes into labels
    @staticmethod
    def map_labels(index):

        labels_dict={
          0: 'Actinic Keratoses and Intraepithelial Carcinoma', # akiec
          1: 'Basal cell carcinoma', # bcc
          2: 'Benign keratosis-like lesions', # bkl
          3: 'Dermatofibroma', # df
          4: 'Melanoma', # mel 
          5: 'Melanocytic nevi', # nv
          6: 'Vascular lesions' # vasc
        } 

        return labels_dict[index] # return with the right value
    
    


### Default models training

In this section 8 different models have been trained with the following parameters:
 - Optimizer: Adam
 - Learning rate: initial learning rate is 0.0001
 - Batch size: 64
 - Max epochs num: 15
 - ReduceLROnPlateau with patience 1
 - EarlyStopping with patience 2
 
The only change in the models architecture is the final fully connected/Dense layer, which contains 7 output instead of 1000. During the training process all layers were frozen.

In [ ]:
# Hyperparameters for training 
lr                = 0.0001
batch_size        = 64
img_size          = 224
epochs_num        = 15
reduce_lr         = True
early_stop        = True
my_skin_cnn       = SkinCancerCnn(img_size, lr, batch_size, reduce_lr, early_stop, epochs_num)

In [ ]:
# VGG16

current_model     = VGG16
units             = 0
num_layers        = 0
dropout           = False
globlayer         = False
freeze_layers_num = 0
model_name        = 'VGG16'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size, multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.vgg16.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn.create_multiclass_generators(X_train,
                                                                                                       y_train,
                                                                                                       multiclass_directories[0],
                                                                                                       multiclass_directories[1],                                       
                                                                                                       multi_sizes,
                                                                                                       tf.keras.applications.vgg16.preprocess_input)

# Training process
vgg16_model, vgg16_history = my_skin_cnn.multi_class_model_train(train_multi_generator,
                                                                 val_multi_generator,
                                                                 multi_val_steps, 
                                                                 current_model,
                                                                 units,
                                                                 num_layers,
                                                                 dropout,
                                                                 globlayer,
                                                                 freeze_layers_num,
                                                                 model_name)
# Plot the training process
SkinCancerCnn.plot_model(vgg16_history, 'plots/default/VGG16_default_plot.jpg')

# Save the model
vgg16_model.save('./new_models/default/VGG16_224x224_batch64_frezze_full_original_0.0001lr.h5')

In [ ]:
# InceptionV3
current_model     = InceptionV3
units             = 0
num_layers        = 0
dropout           = False
globlayer         = False
freeze_layers_num = 0
model_name        = 'InceptionV3'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories,  tf.keras.applications.inception_v3.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn.create_multiclass_generators(X_train,
                                                                                                       y_train,
                                                                                                       multiclass_directories[0],
                                                                                                       multiclass_directories[1],                                       
                                                                                                       multi_sizes,
                                                                                                       tf.keras.applications.inception_v3.preprocess_input)
# Training process
inceptionv3_model, inceptionv3_history = my_skin_cnn.multi_class_model_train(train_multi_generator,
                                                                             val_multi_generator,
                                                                             multi_val_steps, 
                                                                             current_model,
                                                                             units,
                                                                             num_layers,
                                                                             dropout,
                                                                             globlayer,
                                                                             freeze_layers_num,
                                                                             model_name)
# Plot the training process
SkinCancerCnn.plot_model(inceptionv3_history, 'plots/default/InceptionV3_default_plot.jpg')

# Save the model
inceptionv3_model.save('./new_models/default/InceptionV3_224x224_batch64_frezze_full_original_0.0001lr.h5')

In [ ]:
# ResNet50
current_model     = ResNet50
units             = 0
num_layers        = 0
dropout           = False
globlayer         = False
freeze_layers_num = 0
model_name        = 'ResNet50'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.resnet.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn.create_multiclass_generators(X_train,
                                                                                                       y_train,
                                                                                                       multiclass_directories[0],
                                                                                                       multiclass_directories[1],                                       
                                                                                                       multi_sizes,
                                                                                                       tf.keras.applications.resnet.preprocess_input)

# Training process
resnet50_model, resnet50_history = my_skin_cnn.multi_class_model_train(train_multi_generator,
                                                                       val_multi_generator,
                                                                       multi_val_steps, 
                                                                       current_model,
                                                                       units,
                                                                       num_layers,
                                                                       dropout,
                                                                       globlayer,
                                                                       freeze_layers_num,
                                                                       model_name)
# Plot the training process
SkinCancerCnn.plot_model(resnet50_history, 'plots/default/ResNet50_default_plot.jpg')

# Save the model
resnet50_model.save('./new_models/default/ResNet50_224x224_batch64_frezze_full_original_0.0001lr.h5')

In [ ]:
# ResNet152
current_model     = ResNet152
units             = 0
num_layers        = 0
dropout           = False
globlayer         = False
freeze_layers_num = 0
model_name        = 'ResNet152'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size, 
                                                  multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.resnet.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn.create_multiclass_generators(X_train,
                                                                                                       y_train,
                                                                                                       multiclass_directories[0],
                                                                                                       multiclass_directories[1],                                       
                                                                                                       multi_sizes,
                                                                                                       tf.keras.applications.resnet.preprocess_input)

# Training process
resnet152_model, resnet152_history = my_skin_cnn.multi_class_model_train(train_multi_generator,
                                                                         val_multi_generator,
                                                                         multi_val_steps, 
                                                                         current_model,
                                                                         units,
                                                                         num_layers,
                                                                         dropout,
                                                                         globlayer,
                                                                         freeze_layers_num,
                                                                         model_name)
# Plot the training process
SkinCancerCnn.plot_model(resnet152_history, 'plots/default/ResNet152_default_plot.jpg')

# Save the model
resnet152_model.save('./new_models/default/ResNet152_224x224_batch64_frezze_full_original_0.0001lr.h5')

In [ ]:
# DenseNet201
current_model     = DenseNet201
units             = 0
num_layers        = 0
dropout           = False
globlayer         = False
freeze_layers_num = 0
model_name        = 'DenseNet201'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.densenet.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn.create_multiclass_generators(X_train,
                                                                                                       y_train,
                                                                                                       multiclass_directories[0],
                                                                                                       multiclass_directories[1],                                       
                                                                                                       multi_sizes,
                                                                                                       tf.keras.applications.densenet.preprocess_input)

# Training process
densenet201_model, densenet201_history = my_skin_cnn.multi_class_model_train(train_multi_generator,
                                                                             val_multi_generator,
                                                                             multi_val_steps, 
                                                                             current_model,
                                                                             units,
                                                                             num_layers,
                                                                             dropout,
                                                                             globlayer,
                                                                             freeze_layers_num,
                                                                             model_name)

# Plot the training process
SkinCancerCnn.plot_model(densenet201_history, 'plots/default/DenseNet201_default_plot.jpg')

# Save the model
densenet201_model.save('./new_models/default/DenseNet201_224x224_batch64_frezze_full_original_0.0001lr.h5')

In [ ]:
# MobileNetV2
current_model     = MobileNetV2
units             = 0
num_layers        = 0
dropout           = False
globlayer         = False
freeze_layers_num = 0
model_name        = 'MobileNetV2'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.mobilenet_v2.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn.create_multiclass_generators(X_train,
                                                                                                       y_train,
                                                                                                       multiclass_directories[0],
                                                                                                       multiclass_directories[1],                                       
                                                                                                       multi_sizes,
                                                                                                       tf.keras.applications.mobilenet_v2.preprocess_input)

# Training process
mobilenetv2_model, mobilenetv2_history = my_skin_cnn.multi_class_model_train(train_multi_generator,
                                                                             val_multi_generator,
                                                                             multi_val_steps, 
                                                                             current_model,
                                                                             units,
                                                                             num_layers,
                                                                             dropout,
                                                                             globlayer,
                                                                             freeze_layers_num,
                                                                             model_name)

# Plot the training process
SkinCancerCnn.plot_model(mobilenetv2_history, 'plots/default/MobileNetV2_default_plot.jpg')

# Save the model
mobilenetv2_model.save('./new_models/default/MobileNetV2_224x224_batch64_frezze_full_original_0.0001lr.h5')

In [ ]:
# EfficientNetB0
current_model     = EfficientNetB0
units             = 0
num_layers        = 0
dropout           = False
globlayer         = False
freeze_layers_num = 0
model_name        = 'EfficientNetB0'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.efficientnet.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn.create_multiclass_generators(X_train,
                                                                                                       y_train,
                                                                                                       multiclass_directories[0],
                                                                                                       multiclass_directories[1],                                       
                                                                                                       multi_sizes,
                                                                                                       tf.keras.applications.efficientnet.preprocess_input)

# Training process
efficientnetb0_model, efficientnetb0_history = my_skin_cnn.multi_class_model_train(train_multi_generator,
                                                                             val_multi_generator,
                                                                             multi_val_steps, 
                                                                             current_model,
                                                                             units,
                                                                             num_layers,
                                                                             dropout,
                                                                             globlayer,
                                                                             freeze_layers_num,
                                                                             model_name)

# Plot the training process
SkinCancerCnn.plot_model(efficientnetb0_history, 'plots/default/EfficientNetB0_default_plot.jpg')

# Save the model
efficientnetb0_model.save('./new_models/default/EfficientNetB0_224x224_batch64_frezze_full_original_0.0001lr.h5')

In [ ]:
# EfficientNetB7
current_model     = EfficientNetB7
units             = 0
num_layers        = 0
dropout           = False
globlayer         = False
freeze_layers_num = 0
model_name        = 'EfficientNetB7'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0], 
                                                  multi_categories,
                                                  tf.keras.applications.efficientnet.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn.create_multiclass_generators(X_train,
                                                                                                       y_train,
                                                                                                       multiclass_directories[0],
                                                                                                       multiclass_directories[1],                                       
                                                                                                       multi_sizes,
                                                                                                       tf.keras.applications.efficientnet.preprocess_input)

# Training process
efficientnetb7_model, efficientnetb7_history = my_skin_cnn.multi_class_model_train(train_multi_generator,
                                                                                   val_multi_generator,
                                                                                   multi_val_steps, 
                                                                                   current_model,
                                                                                   units,
                                                                                   num_layers,
                                                                                   dropout,
                                                                                   globlayer,
                                                                                   freeze_layers_num,
                                                                                   model_name)

# Plot the training process
SkinCancerCnn.plot_model(efficientnetb7_history, 'plots/default/EfficientNetB7_default_plot.jpg')

# Save the model
efficientnetb7_model.save('./new_models/default/EfficientNetB7_224x224_batch64_frezze_full_original_0.0001lr.h5')

### Optimized models training

In this section 8 different models have been trained with the following parameters:

|Model         |Number of new layers|Number of new neurons per layers| Dropout|Global average pooling|Number of frozen layers|
|--------------|--------------------|--------------------------------|--------|----------------------|-----------------------|
|VGG16         |1                   |512                             |False   |True                  |14                     |
|InceptionV3   |3                   |256                             |False   |True                  |50                     |
|ResNet50      |2                   |384                             |False   |False                 |90                     |
|ResNet152     |3                   |128                             |False   |False                 |200                    |
|DenseNet201   |2                   |384                             |True    |False                 |200                    |
|MobileNetV2   |1                   |256                             |False   |False                 |35                     |
|EfficientNetB0|3                   |384                             |False   |False                 |100                    |
|EfficientNetB7|3                   |128                             |True    |True                  |400                    |

The training process parameters are the followings:
 - Optimizer: Adam
 - Learning rate: 0.00001 initial
 - Batch size: 64
 - Max epochs number: 20
 - ReduceLROnPlateau with patience 2
 - EarlyStopping with patience 3

In [ ]:
# Hyperparameters for training 
lr                = 0.00001
batch_size        = 64
img_size          = 224
epochs_num        = 20
reduce_lr         = True
early_stop        = True
my_skin_cnn_opt   = SkinCancerCnn(img_size, lr, batch_size, reduce_lr, early_stop, epochs_num)

In [ ]:
# VGG16 optimized 

current_model     = VGG16
units             = 512
num_layers        = 1
dropout           = False
globlayer         = True
freeze_layers_num = 14
model_name        = 'VGG16_optimized'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.vgg16.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn_opt.create_multiclass_generators(X_train,
                                                                                                           y_train,
                                                                                                           multiclass_directories[0],
                                                                                                           multiclass_directories[1],                                       
                                                                                                           multi_sizes,
                                                                                                           tf.keras.applications.vgg16.preprocess_input)

# Training process
vgg16_optimized_model, vgg16_optimized_history = my_skin_cnn_opt.multi_class_model_train(train_multi_generator,
                                                                                         val_multi_generator,
                                                                                         multi_val_steps, 
                                                                                         current_model,
                                                                                         units,
                                                                                         num_layers,
                                                                                         dropout,
                                                                                         globlayer,
                                                                                         freeze_layers_num,
                                                                                         model_name)

# Plot the training process
SkinCancerCnn.plot_model(vgg16_optimized_history, 'plots/optimized/VGG16_optimized_plot.jpg')

# Save the model
vgg16_optimized_model.save('./new_models/optimized/VGG16_optimized_224x224_batch64_0.00001lr.h5')

In [ ]:
# InceptionV3 optimized

current_model     = InceptionV3
units             = 256
num_layers        = 3
dropout           = False
globlayer         = True
freeze_layers_num = 50
model_name        = 'InceptionV3_optimized'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.inception_v3.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn_opt.create_multiclass_generators(X_train,
                                                                                                           y_train,
                                                                                                           multiclass_directories[0],
                                                                                                           multiclass_directories[1],                                       
                                                                                                           multi_sizes,
                                                                                                           tf.keras.applications.inception_v3.preprocess_input)

# Training process
inceptionv3_optimized_model, inceptionv3_optimized_history = my_skin_cnn_opt.multi_class_model_train(train_multi_generator,
                                                                                                     val_multi_generator,
                                                                                                     multi_val_steps, 
                                                                                                     current_model,
                                                                                                     units,
                                                                                                     num_layers,
                                                                                                     dropout,
                                                                                                     globlayer,
                                                                                                     freeze_layers_num,
                                                                                                     model_name)

# Plot the training process
SkinCancerCnn.plot_model(inceptionv3_optimized_history, 'plots/optimized/InceptionV3_optimized_plot.jpg')

# Save the model
inceptionv3_optimized_model.save('./new_models/optimized/InceptionV3_optimized_224x224_batch64_0.00001lr.h5')

In [ ]:
# ResNet50 optimized 

current_model     = ResNet50
units             = 384
num_layers        = 2
dropout           = False
globlayer         = False
freeze_layers_num = 90
model_name        = 'ResNet50_optimized'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.resnet.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn_opt.create_multiclass_generators(X_train,
                                                                                                           y_train,
                                                                                                           multiclass_directories[0],
                                                                                                           multiclass_directories[1],                                       
                                                                                                           multi_sizes,
                                                                                                           tf.keras.applications.resnet.preprocess_input)

# Training process
resnet50_optimized_model, resnet50_optimized_history = my_skin_cnn_opt.multi_class_model_train(train_multi_generator,
                                                                                               val_multi_generator,
                                                                                               multi_val_steps, 
                                                                                               current_model,
                                                                                               units,
                                                                                               num_layers,
                                                                                               dropout,
                                                                                               globlayer,
                                                                                               freeze_layers_num,
                                                                                               model_name)

# Plot the training process
SkinCancerCnn.plot_model(resnet50_optimized_history, 'plots/optimized/ResNet50_optimized_plot.jpg')

# Save the model
resnet50_optimized_model.save('./new_models/optimized/ResNet50_optimized_224x224_batch64_0.00001lr.h5')

In [ ]:
# ResNet152 optimized

current_model     = ResNet152
units             = 128
num_layers        = 3
dropout           = False
globlayer         = False
freeze_layers_num = 200
model_name        = 'ResNet152_optimized'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.resnet.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn_opt.create_multiclass_generators(X_train,
                                                                                                           y_train,
                                                                                                           multiclass_directories[0],
                                                                                                           multiclass_directories[1],                                       
                                                                                                           multi_sizes,
                                                                                                           tf.keras.applications.resnet.preprocess_input)

# Training process
resnet152_optimized_model, resnet152_optimized_history = my_skin_cnn_opt.multi_class_model_train(train_multi_generator,
                                                                                                 val_multi_generator,
                                                                                                 multi_val_steps, 
                                                                                                 current_model,
                                                                                                 units,
                                                                                                 num_layers,
                                                                                                 dropout,
                                                                                                 globlayer,
                                                                                                 freeze_layers_num,
                                                                                                 model_name)

# Plot the training process
SkinCancerCnn.plot_model(resnet152_optimized_history, 'plots/optimized/ResNet152_optimized_plot.jpg')

# Save the model
resnet152_optimized_model.save('./new_models/optimized/ResNet152_optimized_224x224_batch64_0.00001lr.h5')

In [ ]:
# DenseNet201 optimized

current_model     = DenseNet201
units             = 384
num_layers        = 2
dropout           = True
globlayer         = False
freeze_layers_num = 200
model_name        = 'DenseNet201_optimized'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories,
                                                  tf.keras.applications.densenet.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn_opt.create_multiclass_generators(X_train,
                                                                                                           y_train,
                                                                                                           multiclass_directories[0],
                                                                                                           multiclass_directories[1],                                       
                                                                                                           multi_sizes,
                                                                                                           tf.keras.applications.densenet.preprocess_input)

# Training process
densenet201_optimized_model, densenet201_optimized_history = my_skin_cnn_opt.multi_class_model_train(train_multi_generator,
                                                                                                     val_multi_generator,
                                                                                                     multi_val_steps, 
                                                                                                     current_model,
                                                                                                     units,
                                                                                                     num_layers,
                                                                                                     dropout,
                                                                                                     globlayer,
                                                                                                     freeze_layers_num,
                                                                                                     model_name)

# Plot the training process
SkinCancerCnn.plot_model(densenet201_optimized_history, 'plots/optimized/DenseNet201_optimized_plot.jpg')

# Save the model
densenet201_optimized_model.save('./new_models/optimized/DenseNet201_optimized_224x224_batch64_0.00001lr.h5')

In [ ]:
# MobileNetV2 optimized

current_model     = MobileNetV2
units             = 256
num_layers        = 1
dropout           = False
globlayer         = False
freeze_layers_num = 35
model_name        = 'MobileNetV2_optimized'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0], 
                                                  multi_categories,
                                                  tf.keras.applications.mobilenet_v2.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn_opt.create_multiclass_generators(X_train,
                                                                                                           y_train,
                                                                                                           multiclass_directories[0],
                                                                                                           multiclass_directories[1],                                       
                                                                                                           multi_sizes,
                                                                                                           tf.keras.applications.mobilenet_v2.preprocess_input)

# Training process
mobilenetv2_optimized_model, mobilenetv2_optimized_history = my_skin_cnn_opt.multi_class_model_train(train_multi_generator,
                                                                                                     val_multi_generator,
                                                                                                     multi_val_steps, 
                                                                                                     current_model,
                                                                                                     units,
                                                                                                     num_layers,
                                                                                                     dropout,
                                                                                                     globlayer,
                                                                                                     freeze_layers_num,
                                                                                                     model_name)

# Plot the training process
SkinCancerCnn.plot_model(mobilenetv2_optimized_history, 'plots/optimized/MobileNetV2_optimized_plot.jpg')

# Save the model
mobilenetv2_optimized_model.save('./new_models/optimized/MobileNetV2_optimized_224x224_batch64_0.00001lr.h5')

In [ ]:
# EfficientNetB0 optimized

current_model     = EfficientNetB0
units             = 384
num_layers        = 3
dropout           = False
globlayer         = False
freeze_layers_num = 100
model_name        = 'EfficientNetB0_optimized'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size, 
                                                  multiclass_directories[0], 
                                                  multi_categories,
                                                  tf.keras.applications.efficientnet.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn_opt.create_multiclass_generators(X_train,
                                                                                                           y_train,
                                                                                                           multiclass_directories[0],
                                                                                                           multiclass_directories[1],                                       
                                                                                                           multi_sizes,
                                                                                                           tf.keras.applications.efficientnet.preprocess_input)

# Training process
efficientnetb0_optimized_model, efficientnetb0_optimized_history = my_skin_cnn_opt.multi_class_model_train(train_multi_generator,
                                                                                                           val_multi_generator,
                                                                                                           multi_val_steps, 
                                                                                                           current_model,
                                                                                                           units,
                                                                                                           num_layers,
                                                                                                           dropout,
                                                                                                           globlayer,
                                                                                                           freeze_layers_num,
                                                                                                           model_name)

# Plot the training process
SkinCancerCnn.plot_model(efficientnetb0_optimized_history, 'plots/optimized/EfficientNetB0_optimized_plot.jpg')

# Save the model
efficientnetb0_optimized_model.save('./new_models/optimized/EfficientNetB0_optimized_224x224_batch64_0.00001lr.h5')

In [ ]:
# EfficientNetB7 optimized

current_model     = EfficientNetB7
units             = 128
num_layers        = 3
dropout           = True
globlayer         = True
freeze_layers_num = 400
model_name        = 'EfficientNetB7_optimized'

# Create numpy array and generators for the model
X_train, y_train = SkinCancerCnn.create_image_arr(img_size, 
                                                  multiclass_directories[0], 
                                                  multi_categories,
                                                  tf.keras.applications.efficientnet.preprocess_input,
                                                  'train')

train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn_opt.create_multiclass_generators(X_train,
                                                                                                           y_train,
                                                                                                           multiclass_directories[0],
                                                                                                           multiclass_directories[1],                                       
                                                                                                           multi_sizes,
                                                                                                           tf.keras.applications.efficientnet.preprocess_input)

# Training process
efficientnetb7_optimized_model, efficientnetb7_optimized_history = my_skin_cnn_opt.multi_class_model_train(train_multi_generator,
                                                                                                           val_multi_generator,
                                                                                                           multi_val_steps, 
                                                                                                           current_model,
                                                                                                           units,
                                                                                                           num_layers,
                                                                                                           dropout,
                                                                                                           globlayer,
                                                                                                           freeze_layers_num,
                                                                                                           model_name)

# Plot the training process
SkinCancerCnn.plot_model(efficientnetb7_optimized_history, 'plots/optimized/EfficientNetB7_optimized_plot.jpg')

# Save the model
efficientnetb7_optimized_model.save('./new_models/optimized/EfficientNetB7_optimized_224x224_batch64_0.00001lr.h5')

### Hyperparameter search

This section was used for Bayesian hyperparameter optimization. This is a long process because it tried different model configurations. On average it took about 15 hours/model.

In [ ]:
# Hyperparameters for training 
lr                = 0.00001
batch_size        = 64
img_size          = 224
epochs_num        = 13
reduce_lr         = True
early_stop        = True
my_skin_cnn_hyper = SkinCancerCnn(img_size, lr, batch_size, reduce_lr, early_stop, epochs_num)


# Create numpy array and generators for Multiclass
X_train, y_train = SkinCancerCnn.create_image_arr(img_size,
                                                  multiclass_directories[0],
                                                  multi_categories, 
                                                  tf.keras.applications.efficientnet.preprocess_input, 
                                                  'train')

# Multi class NN
train_multi_generator, val_multi_generator, multi_val_steps = my_skin_cnn_hyper.create_multiclass_generators(X_train,
                                                                                                             y_train,
                                                                                                             multiclass_directories[0],
                                                                                                             multiclass_directories[1],                                       
                                                                                                             multi_sizes,
                                                                                                             tf.keras.applications.efficientnet.preprocess_input)


In [ ]:
def build_model(hp):
    
    current_model     = EfficientNetB0
    units             = hp.Int("units", min_value = 128, max_value = 512, step = 128)
    num_layers        = hp.Int("layers", min_value = 1, max_value = 3)
    dropout           = hp.Boolean("dropout")
    globlayer         = hp.Boolean("globlayer")
    freeze_layers_num = hp.Int("freeze_layers_num", min_value = 0, max_value = 200, step = 50)
    
    # call existing model-building code with the hyperparameter values.
    model = my_skin_cnn_hyper.multi_class_model(train_multi_generator,
                                                val_multi_generator,
                                                multi_val_steps,
                                                current_model,
                                                units,
                                                num_layers,
                                                dropout,
                                                globlayer,
                                                freeze_layers_num)

    return model

In [ ]:
build_model(kt.HyperParameters())

In [ ]:
tuner = kt.BayesianOptimization(build_model,
                                max_trials = 15, 
                                # Do not resume the previous search in the same directory.
                                overwrite = True,
                                objective = kt.Objective("val_recall", direction=  "max"),
                                # Set a directory to store the intermediate results.
                                directory = "my_dir")


callbacks = []


decrease_lr = ReduceLROnPlateau(monitor   = 'val_recall',
                                factor    = 0.2,
                                min_lr    = 0.0000001,
                                patience  = 2,
                                verbose   = 1,
                                min_delta = 1e-6,
                                mode      = 'max')

early_stop = EarlyStopping(monitor              = 'val_recall',
                           min_delta            = 0,
                           patience             = 3,
                           verbose              = 0,
                           mode                 = 'max',
                           baseline             = None,
                           restore_best_weights = True)

callbacks.append(decrease_lr)
callbacks.append(early_stop)
callbacks.append(keras.callbacks.TensorBoard("loggings_bayes_efficientnetb0"))    

tuner.search(train_multi_generator,
             callbacks        = callbacks,
             epochs           = 13, 
             validation_data  = val_multi_generator,
             validation_steps = multi_val_steps)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete.
The optimal number of units is {best_hps.get('units')} 
and the optimal number of layers is {best_hps.get('layers')}
and the optimal use of dropout is {best_hps.get('dropout')}
and the optimal globlayer is {best_hps.get('globlayer')}
and the optimal freeze_layers_num is {best_hps.get('freeze_layers_num')}.
""")

In [ ]:
# Optimized hyperparameters for NN
lr                = 0.00001
batch_size        = 64
units             = best_hps.get('units')
num_layers        = best_hps.get('layers')         
dropout           = best_hps.get('dropout')
globlayer         = best_hps.get('globlayer')
freeze_layers_num = best_hps.get('freeze_layers_num')

# Save the hyperparameters
with open('./hyperparameters/hyperparameters_effnet.txt', 'w') as file:
    file.write('lr ' + str(lr) + '\n')
    file.write('batch_size ' + str(batch_size) + '\n')
    file.write('units ' + str(units) + '\n')
    file.write('num_layers ' + str(num_layers) + '\n')
    file.write('dropout ' + str(dropout) + '\n')
    file.write('globlayer ' + str(globlayer) + '\n')
    file.write('freeze_layers_num ' + str(freeze_layers_num) + '\n')

### Loading the default models

In this section, we are loading the trained default models for evaluation on the test set. We save the metrics scores, heatmaps with counter, and heatmaps with normalized values. The metrics we are interested in are the followings:
 - Accuracy
 - Precision
 - Recall / Balanced accuracy
 - F1-score
 - MCC
 - AVG
 
We save the ROC-AUC curve for the best model. 

In [ ]:
img_size = 224

In [ ]:
# VGG16
model_vgg16 = keras.models.load_model('./models/default models/VGG16_224x224_batch64_frezze_full_original_0.0001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories, tf.keras.applications.vgg16.preprocess_input, 'test') # create an image array with their corresponding labels for the test images
y_pred = model_vgg16.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/default/VGG16_default_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/default/VGG16_default_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/default/VGG16_default_heatmap_normalized.jpg')

In [ ]:
# InceptionV3
model_inceptionv3 = keras.models.load_model('./models/default models/InceptionV3_224x224_batch64_frezze_full_original_0.0001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories, tf.keras.applications.inception_v3.preprocess_input, 'test') # create an image array with their corresponding labels for the test images
y_pred = model_inceptionv3.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/default/InceptionV3_default_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/default/InceptionV3_default_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/default/InceptionV3_default_heatmap_normalized.jpg')

In [ ]:
# ResNet50
model_resnet50 = keras.models.load_model('./models/default models/ResNet50_224x224_batch64_frezze_full_original_0.0001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories, tf.keras.applications.resnet.preprocess_input, 'test') # create an image array with their corresponding labels for the test images
y_p = model_resnet50.predict(X_test)
y_pred = [np.argmax(element) for element in y_p]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/default/ResNet50_default_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/default/ResNet50_default_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/default/ResNet50_default_heatmap_normalized.jpg')

In [ ]:
# ResNet50 ROC-AUC curve
SkinCancerCnn.plot_ROC_AUC_curve(y_true, y_p, 'ROC-AUC/default/ResNet50_default_ROC_AUC.jpg') 

In [ ]:
# ResNet152
model_resnet152 = keras.models.load_model('./models/default models/ResNet152_224x224_batch64_frezze_full_original_0.0001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.resnet.preprocess_input, 'test') # create an image array with their corresponding labels for the test images
y_pred = model_resnet152.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/default/ResNet152_default_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/default/ResNet152_default_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/default/ResNet152_default_heatmap_normalized.jpg')

In [ ]:
# DenseNet201
model_densenet201 = keras.models.load_model('./models/default models/DenseNet201_224x224_batch64_frezze_full_original_0.0001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.densenet.preprocess_input, 'test') # create an image array with their corresponding labels for the test images
y_pred = model_densenet201.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/default/DenseNet201_default_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/default/DenseNet201_default_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/default/DenseNet201_default_heatmap_normalized.jpg')

In [ ]:
# MobileNetV2
model_mobilenetv2 = keras.models.load_model('./models/default models/MobileNetV2_224x224_batch64_frezze_full_original_0.0001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.mobilenet_v2.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_pred = model_mobilenetv2.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/default/MobileNetV2_default_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/default/MobileNetV2_default_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/default/MobileNetV2_default_heatmap_normalized.jpg')

In [ ]:
# EfficientNetB0
model_efficientnetb0 = keras.models.load_model('./models/default models/EfficientNetB0_224x224_batch64_frezze_full_original_0.0001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.efficientnet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_pred = model_efficientnetb0.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/default/EfficientNetB0_default_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/default/EfficientNetB0_default_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/default/EfficientNetB0_default_heatmap_normalized.jpg')

In [ ]:
# EfficientNetB7
model_efficientnetb7 = keras.models.load_model('./models/default models/EfficientNetB7_224x224_batch64_frezze_full_original_0.0001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.efficientnet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_pred = model_efficientnetb7.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/default/EfficientNetB7_default_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/default/EfficientNetB7_default_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/default/EfficientNetB7_default_heatmap_normalized.jpg')

### Loading the optimized models

The procedure in this section is the same as in the 'default' models case. 

In [ ]:
img_size = 224

In [ ]:
# VGG16_optimized
model_vgg16_optimized = keras.models.load_model('./models/optimized models/VGG16_optimized_224x224_batch64_0.00001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.vgg16.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_pred = model_vgg16_optimized.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/optimized/VGG16_optimized_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/optimized/VGG16_optimized_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/optimized/VGG16_optimized_heatmap_normalized.jpg')

In [ ]:
# InceptionV3_optimized
model_inceptionv3_optimized = keras.models.load_model('./models/optimized models/InceptionV3_optimized_224x224_batch64_0.00001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.inception_v3.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_pred = model_inceptionv3_optimized.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/optimized/InceptionV3_optimized_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/optimized/InceptionV3_optimized_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/optimized/InceptionV3_optimized_heatmap_normalized.jpg')

In [ ]:
# ResNet50_optimized
model_resnet50_optimized = keras.models.load_model('./models/optimized models/ResNet50_optimized_224x224_batch64_0.00001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.resnet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_pred = model_resnet50_optimized.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/optimized/ResNet50_optimized_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/optimized/ResNet50_optimized_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/optimized/ResNet50_optimized_heatmap_normalized.jpg')

In [ ]:
# ResNet152_optimized
model_resnet152_optimized = keras.models.load_model('./models/optimized models/ResNet152_optimized_224x224_batch64_0.00001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.resnet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_pred = model_resnet152_optimized.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/optimized/ResNet152_optimized_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/optimized/ResNet152_optimized_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/optimized/ResNet152_optimized_heatmap_normalized.jpg')

In [ ]:
# DenseNet201_optimized
model_densenet201_optimized = keras.models.load_model('./models/optimized models/DenseNet201_optimized_224x224_batch64_0.00001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.densenet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_pred = model_densenet201_optimized.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/optimized/DenseNet201_optimizedmetrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/optimized/DenseNet201_optimized_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/optimized/DenseNet201_optimized_heatmap_normalized.jpg')

In [ ]:
# MobileNetV2_optimized
model_mobilenetv2_optimized = keras.models.load_model('./models/optimized models/MobileNetV2_optimized_224x224_batch64_0.00001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.mobilenet_v2.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_pred = model_mobilenetv2_optimized.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/optimized/MobileNetV2_optimized_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/optimized/MobileNetV2_optimized_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/optimized/MobileNetV2_optimized_heatmap_normalized.jpg')

In [ ]:
# EfficientNetB0_optimized
model_efficientnetb0_optimized = keras.models.load_model('./models/optimized models/EfficientNetB0_optimized_224x224_batch64_0.00001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.efficientnet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_pred = model_efficientnetb0_optimized.predict(X_test)
y_pred = [np.argmax(element) for element in y_pred]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/optimized/EfficientNetB0_optimized_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/optimized/EfficientNetB0_optimized_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/optimized/EfficientNetB0_optimized_heatmap_normalized.jpg')

In [ ]:
# EfficientNetB7_optimized
model_efficientnetb7_optimized = keras.models.load_model('./models/optimized models/EfficientNetB7_optimized_224x224_batch64_0.00001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.efficientnet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
y_p = model_efficientnetb7_optimized.predict(X_test)
y_pred = [np.argmax(element) for element in y_p]

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/optimized/EfficientNetB7_optimized_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, None, 'heatmaps/optimized/EfficientNetB7_optimized_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/optimized/EfficientNetB7_optimized_heatmap_normalized.jpg')

In [ ]:
# EfficientNetB7_optimized ROC-AUC curve
SkinCancerCnn.plot_ROC_AUC_curve(y_true, y_p, 'ROC-AUC/optimized/EfficientNetB7_optimized_ROC_AUC.jpg') 

### Weighted average ensemble model

In this section, we took the 3 best models and use them for prediction. After the prediction phase we multiple that with the best weights for the ensemble predictions. In the last step, we save the metrics, confusion matrices, and ROC-AUC curve.

#### Grid search for optimal weights

In [ ]:
# Grid search for weights

def best_weights_grid_search(preds):
    max_avg = 0.0 

    w1_opt = 0.0
    w2_opt = 0.0
    w3_opt = 0.0

    for w1 in range(0,9):
        for w2 in range(0,9):
            for w3 in range(0,9):

                wts                = [w1/10., w2/10., w3/10.]
                wted_preds1        = np.tensordot(preds, wts, axes=((0),(0)))
                wted_ensemble_pred = np.argmax(wted_preds1, axis=1)
                accuracy           = accuracy_score(y_true, wted_ensemble_pred) # How many we got right
                recall             = recall_score(y_true, wted_ensemble_pred, average = 'macro') # Truth labels are the baseline, All class[i] truth how many we got right
                precision          = precision_score(y_true, wted_ensemble_pred, average = 'macro') # All class[i] predictions how many we got right
                f1_score           = 2 * (precision * recall) / (precision + recall)
                balanced_accuracy  = balanced_accuracy_score(y_true, wted_ensemble_pred)
                mcc                = matthews_corrcoef(y_true, wted_ensemble_pred)

                AVG = (balanced_accuracy + f1_score + mcc) / 3.0

                if AVG > max_avg:
                    max_avg = AVG
                    w1_opt = w1 / 10.0
                    w2_opt = w2 / 10.0
                    w3_opt = w3 / 10.0

    print('Best weights: ' + str(w1_opt) + ' ' + str(w2_opt) + ' ' + str(w3_opt) + ' with AVG: ' + str(max_avg)) 
    return [w1_opt, w2_opt, w3_opt]

Loading the three models and make predictions

In [ ]:
img_size = 224

In [ ]:
model_efficientnetb7_ensemble = keras.models.load_model('./models/optimized models/EfficientNetB7_optimized_224x224_batch64_0.00001lr.h5')
model_resnet50_ensemble       = keras.models.load_model('./models/optimized models/ResNet50_optimized_224x224_batch64_0.00001lr.h5')
model_inceptionv3_ensemble    = keras.models.load_model('./models/optimized models/InceptionV3_optimized_224x224_batch64_0.00001lr.h5')

X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.efficientnet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
X_test2,_      = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.resnet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
X_test3,_      = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.inception_v3.preprocess_input,  'test') # create an image array with their corresponding labels for the test images


pred_efficientnet = model_efficientnetb7_ensemble.predict(X_test)
pred_resnet       = model_resnet50_ensemble.predict(X_test2)
pred_inception    = model_inceptionv3_ensemble.predict(X_test3)
preds = [pred_efficientnet, pred_resnet, pred_inception]
preds = np.array(preds)

# Weighted
#weights = [0.5, 0.7, 0.4]
weights = best_weights_grid_search(preds)

# Use tensordot to sum the products of all elements over specified axes.
weighted_preds = np.tensordot(preds, weights, axes=((0),(0)))
weighted_ensemble_prediction = np.argmax(weighted_preds, axis=1)

# Metrics, confusion matrix, normalized confusion matrix
SkinCancerCnn.metrics_scores(y_true, weighted_ensemble_prediction, 'metrics/ensemble/ensemble_metrics.txt')
SkinCancerCnn.confusion_matrix(y_true, weighted_ensemble_prediction, 'true', 'heatmaps/ensemble/ensemble_heatmap.jpg')
SkinCancerCnn.confusion_matrix(y_true, weighted_ensemble_prediction, None, 'heatmaps/ensemble/ensemble_heatmap_normalized.jpg')

In [ ]:
# ROC-AUC curve
SkinCancerCnn.plot_ROC_AUC_curve(y_true, weighted_preds, 'ROC-AUC/ensemble/ensemble_ROC_AUC.jpg') 

### Plots for metrics

In this section, I created the plots for the default, optimized, and the ensemble model. I created one plot for the accuracies and one for the AVG metrics.

In [ ]:
def metrics_visualization(models, values, fname, red):

    plt.rcParams["figure.figsize"] = (13,7)

    x  = np.arange(len(models))
    y  = values

    width   = 0.7
    fig, ax = plt.subplots()

    if red:
        pps = ax.bar(x, y, width, align='center', color = (0.8, 0.1, 0.1, 0.8))
    else:
        pps = ax.bar(x, y, width, align='center')
        
    plt.xticks(x, models)

    for p in pps:
        height = p.get_height()
        ax.text(x=p.get_x() + p.get_width() / 2, y=height - 9,
          s="{}%".format(height),
          ha='center', fontsize=14, color = 'white')

    plt.show()
    fig.savefig(fname)


In [ ]:
models_default_acc = ['DenseNet201', 'ResNet50', 'EfficientNetB0', 'ResNet152', 'EfficientNetB7', 'VGG16', 'InceptionV3', 'MobileNetV2']
default_acc        = [77.53, 77.53, 74.34, 73.80, 73.14, 72.47, 70.21, 67.02]
models_default_avg = ['ResNet50', 'ResNet152', 'DenseNet201', 'EfficientNetB7', 'EfficientNetB0', 'VGG16', 'MobileNetV2', 'InceptionV3']
default_avg        = [65.90, 65.53, 63.59, 61.50, 62.08, 58.79, 56.76, 55.48]

models_optimized_acc = ['ResNet50', 'EfficientNetB7',  'InceptionV3', 'EfficientNetB0', 'ResNet152', 'VGG16','DenseNet201', 'MobileNetV2']
optimized_acc        = [86.30, 85.77, 85.77, 84.57, 83.78, 83.78, 81.25, 76.06]
models_optimized_avg = ['EfficientNetB7', 'ResNet50', 'InceptionV3', 'ResNet152', 'EfficientNetB0', 'DenseNet201', 'VGG16', 'MobileNetV2']
optimized_avg        = [78.95, 78.88, 77.20, 76.14, 75.13, 73.01, 72.54, 63.88]

models_ensemble_avg = ['eERI', 'EfficientNetB7', 'ResNet50', 'InceptionV3', 'ResNet152', 'EfficientNetB0', 'DenseNet201', 'VGG16', 'MobileNetV2']
ensemble_avg        = [84.49, 78.95, 78.88, 77.20, 76.14, 75.13, 73.01, 72.54, 63.88]

metrics_visualization(models_default_acc, default_acc, 'metrics_visualization/default/default_acc.jpg', True)
metrics_visualization(models_default_avg, default_avg, 'metrics_visualization/default/default_avg.jpg', False)
metrics_visualization(models_optimized_acc, optimized_acc, 'metrics_visualization/optimized/optimized_acc.jpg', True)
metrics_visualization(models_optimized_avg, optimized_avg, 'metrics_visualization/optimized/optimized_avg.jpg', False)
metrics_visualization(models_ensemble_avg, ensemble_avg, 'metrics_visualization/ensemble/ensemble_avg.jpg', False)

### Gaussian and Poisson noise

In [ ]:
img_size = 224

I created new folders (new test set) for the Poisson and Gaussian noise images. After that, I tested the best 'eERI' model with these images. 

In [ ]:
def create_perturbated_folders(path, mode):

    directories = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'] 
    images_path = []

    for directory in directories:
        for img in os.listdir(path + '/' + directory):
            images_path.append(os.path.join(path,directory,img))


    for img_path in images_path:
        img = skimage.io.imread(img_path)
        gimg = skimage.util.random_noise(img, mode=mode)
        skimage.io.imsave(img_path, gimg)

These folders are already there!

In [ ]:
'''poisson_path  = 'D:\\MSC_szakdolgozat\\MSC_szakdoga\\Datas\\Multiclass\\Poisson test set'
gaussian_path = 'D:\\MSC_szakdolgozat\\MSC_szakdoga\\Datas\\Multiclass\\Gaussian test set'

create_perturbated_folders(poisson_path, 'poisson')
create_perturbated_folders(gaussian_path, 'gaussian')'''

In [ ]:
def adversarial_black_box_perturbation(path, perturbation, mode):

    model_efficientnetb7 = keras.models.load_model('./models/optimized models/EfficientNetB7_optimized_224x224_batch64_0.00001lr.h5')
    model_resnet50       = keras.models.load_model('./models/optimized models/ResNet50_optimized_224x224_batch64_0.00001lr.h5')
    model_inceptionv3    = keras.models.load_model('./models/optimized models/InceptionV3_optimized_224x224_batch64_0.00001lr.h5')

    multi_test = './Datas/Multiclass/' + path

    X_test, y_true = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.efficientnet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
    X_test2,_      = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.resnet.preprocess_input,  'test') # create an image array with their corresponding labels for the test images
    X_test3,_      = SkinCancerCnn.create_image_arr(img_size, multi_test, multi_categories,  tf.keras.applications.inception_v3.preprocess_input,  'test') # create an image array with their corresponding labels for the test images

    pred_efficientnet = model_efficientnetb7.predict(X_test)
    pred_resnet       = model_resnet50.predict(X_test2)
    pred_inception    = model_inceptionv3.predict(X_test3)
    preds = [pred_efficientnet, pred_resnet, pred_inception]
    preds = np.array(preds)

    #weighted average
    weights = [0.5, 0.7, 0.4]

    #Use tensordot to sum the products of all elements over specified axes.
    weighted_preds               = np.tensordot(preds, weights, axes=((0),(0)))
    weighted_ensemble_prediction = np.argmax(weighted_preds, axis=1)

    SkinCancerCnn.metrics_scores(y_true, weighted_ensemble_prediction, 'metrics/' + mode + '/' + perturbation + '.txt')
    SkinCancerCnn.confusion_matrix(y_true, weighted_ensemble_prediction, 'true', 'heatmaps/perturbation/' + perturbation + '.jpg')

In [ ]:
adversarial_black_box_perturbation('Poisson test set', 'Possion_perturbation_eERI', 'poisson')

In [ ]:
adversarial_black_box_perturbation('Gaussian test set', 'Gaussian_perturbation_eERI', 'gauss')

This code section was used for CNN confidence level check with Poisson and Gaussian noise images.

In [ ]:
def confidence_visualization(model_origin, model_poisson, model_gauss, fname):
    
    plt.rcParams["figure.figsize"] = (12,8)
    labels = [ 'akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

    x       = np.arange(len(labels))  # the label locations
    width   = 0.28  # the width of the bars
    fig, ax = plt.subplots()

    bar1 = ax.bar(x, model_origin, width, color = 'r', label = 'eERI')
    bar2 = ax.bar(x+width, model_poisson, width, color='g', label = 'eERI Poisson')
    bar3 = ax.bar(x+width*2, model_gauss, width, color = 'b', label = 'eERI Gaussian')


    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_xticks(x, labels)
    ax.legend()

    ax.bar_label(bar1, padding=3)
    ax.bar_label(bar2, padding=3)
    ax.bar_label(bar3, padding=3)

    fig.tight_layout()
    fig.savefig(fname)


    plt.show()

In [ ]:
# This values were the confidence levels for the given images (for more information see the .pdf file)
model_origin  = [0.03, 0.08, 0.17, 0.05, 87.27, 12.36, 0.05]
model_poisson = [0.01, 0.04, 0.19, 0.08, 60.54, 39.09, 0.05]
model_gauss   = [0.01, 0.03, 0.71, 0.08, 14.96, 84.15, 0.07]

confidence_visualization(model_origin, model_poisson, model_gauss, 'confidence.jpg')

### FGSM white box untargeted attack

I tested the optimized EfficientNetB7 and ResNet50 models with white box untargeted FGSM attacks.

In [ ]:
model_efficientnetb7 = keras.models.load_model('./models/optimized models/EfficientNetB7_optimized_224x224_batch64_0.00001lr.h5')
model_resnet50       = keras.models.load_model('./models/optimized models/ResNet50_optimized_224x224_batch64_0.00001lr.h5')

#### Helper functions for FGSM

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

def preprocess(image, preprocess, img_size):
    
    image = tf.image.resize(image, (img_size, img_size))
    image = preprocess(image)
    image = image[None, ...]
    return image

def create_adversarial_pattern(input_image, input_label, model):
    
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model(input_image)
        loss = loss_object(input_label, prediction)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad

def mapping_labels(index):
    
    labels_dict={
      0: 'akiec', 
      1: 'bcc', 
      2: 'bkl', 
      3: 'df', 
      4: 'mel',
      5: 'nv', 
      6: 'vasc' 
    } 
    
    return labels_dict[index]

def mapping_indices(index):
    
    labels_dict={
      'akiec' : 0, 
      'bcc': 1, 
      'bkl': 2, 
      'df': 3, 
      'mel': 4, 
      'nv': 5, 
      'vasc': 6 
    } 
    
    return labels_dict[index]

def display_images(image, model):
    
    predictions = model.predict(image)
    label = np.argmax(predictions, axis = 1)
    confidence = float(max((model.predict(image))[0]))
    #plt.figure()
    #plt.imshow(image[0]  / 255.0) #*0.5+0.5
    #plt.imshow(image[0] *0.5+0.5) #*0.5+0.5
    #print(label)
    #plt.title('\n {} : {:.2f}% Confidence'.format(mapping_labels(label[0]), confidence*100))
    #plt.show()
    return label

Reading images from the original Test set, and adding perturbation for the images one by one iteratively

In [ ]:
def FGSM(img_size, model, prep, epsilons):

    path        = './Datas/Multiclass/Test set'
    directories = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'] 
    y_true      = []
    y_pred      = []
    ind         = 0

    for directory in directories:
        for img in os.listdir(path + '/' + directory):

            img_path  = os.path.join(path,directory,img)
            image_raw = tf.io.read_file(img_path)
            image     = tf.image.decode_image(image_raw)

            image       = preprocess(image, prep, img_size)
            image_probs = model.predict(image)
            label       = np.argmax(image_probs, axis = 1)

            skin_index = mapping_indices(directory)
            y_true.append(skin_index)

            label      = tf.one_hot(skin_index, image_probs.shape[-1])
            label      = tf.reshape(label, (1, image_probs.shape[-1]))

            perturbations = create_adversarial_pattern(image, label, model)

            epsilons = epsilons

            for i, eps in enumerate(epsilons):
                adv_x = image + eps*perturbations            
                pred_label = display_images(adv_x, model)
                y_pred.append(pred_label[0])
            ind += 1
            print('The current image is: ' + str(ind))
            
    return y_true, y_pred


In [ ]:
#y_true, y_pred = FGSM(224, model_efficientnetb7, tf.keras.applications.efficientnet.preprocess_input, [0.1])
#y_true, y_pred = FGSM(224, model_efficientnetb7, tf.keras.applications.efficientnet.preprocess_input, [0.3])
#y_true, y_pred = FGSM(224, model_efficientnetb7, tf.keras.applications.efficientnet.preprocess_input, [0.5])

In [ ]:
y_true, y_pred = FGSM(224, model_resnet50, tf.keras.applications.resnet.preprocess_input, [0.5])

In [ ]:
y_t = np.array(y_true)
y_p = np.array(y_pred)

SkinCancerCnn.metrics_scores(y_true, y_pred, 'metrics/FGSM/ResNet50_fgsm_metrics_0_5.txt')
SkinCancerCnn.confusion_matrix(y_true, y_pred, 'true', 'heatmaps/perturbation/ResNet50_fgsm_conf_mat_0_5.png')